In [1]:
import torch
import time
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

In [2]:
def init_model():
    model = AutoModelForCausalLM.from_pretrained(
        "/data/tds/ChatGLM3/chatglm3-6b",
        torch_dtype=torch.float16,
        device_map="cuda:1",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "/data/tds/ChatGLM3/chatglm3-6b",
        use_fast=False,
        trust_remote_code=True
    )
    return model, tokenizer


model, tokenizer = init_model()
model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

In [3]:
#多轮对话
prompt1 = """
你是一位精通营销知识和文案策划的小红书爆款写作专家，您的任务通过学习样例文案的语言风格，创作出满足以下要求的小红书文风内容，你不允许回答“好的，当然可以”等描述性词语，也不允许回答“标题，正文”等格式性内容，请勿直接照抄样例文案，你只需要将创作好的内容返回给用户即可，
               要求：  
                   主题：推荐JOVS品牌的美容仪
                   品牌卖点： 家用、深层激活、弹走顽固深纹、双重实力、充盈肌肤
                   受众群体：开始有抗初老需求的年轻女性：如“80后”、“90后”等
                   平台文风：小红书
                   使用场景：作为员工福利赠送
                   推荐角度：请站在职场女性的视角推荐产品
                   文案风格：请以专业风格，突出美容仪的专业性和效果的语言阐述 
                   故事性：请加入故事性描写，以引发并留住读者的兴趣 
                   字数要求：250字 
                   营销目的：吸引对美容有专业要求的消费者 
                   写作要求：使用惊叹号、表情符号等增加文章活力，运用悬念来引发读者好奇心
                   其他要求：在文案中强调美容仪的品质保证和售后服务，让消费者放心购买和使用 
               样例文案：
               ```
               90后的姐妹还没开始抗初老的，请告诉我好吗，我带你在家躺着抗初老。 还在观望的姐妹，可以直接入我的这个同款了，就是这个☞JOVS 4D聚能淡纹面罩，亲测效果明显，是真的可以淡细纹的。 我这才坚持用了两1个月，效果就满那么明显，你们也可以的，还有千万不要等到形成真性纹垮脸了再去考虑抗老，那就太晚太晚了，而且我现在用的这个面罩美容仪，使用的时候也不耽误刷视频看小红书，不漏光不刺眼，而且像是眼角眼袋都能照射到，相当奈斯~ #jovs   #早衰   #美容仪90后的姐妹还有没开始抗初老的吗？
               ```
"""
response, history = model.chat(tokenizer, prompt1, history=[])
print(response)

亲们，你们知道吗？当我第一次听说JOVS品牌的4D聚能淡纹面罩时，我简直不敢相信！这是真的可以淡细纹吗？作为一个90后的小仙女，我当然不能忽视初老问题。于是，我毅然决然地入手了这款JOVS 4D聚能淡纹面罩，用了一段时间后，效果真的让我惊艳！你们也一定要试试看呀！

不要等形成真性纹垮脸了再去考虑抗老，那真的太晚了！我现在用的这个面罩美容仪，使用的时候也不耽误刷视频看小红书，不漏光不刺眼，而且像是眼角眼袋都能照射到，简直太奈斯了！你们一定要抓住这个机会，让JOVS陪伴你们度过初老岁月，让我们一起成为美丽 women！#jovs #抗初老 #美容仪 #4D聚能淡纹面罩


In [11]:
#多轮对话
prompt1 = """
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类": ,"产品品牌": ,"产品特点": ,"受众群体":}
帮我写个文案，小红书风格的，帕兰朵品牌的内衣，小胸聚拢，给精致妈妈人群
"""
response, history = model.chat(tokenizer, prompt1, history=[])
print(response)

好的，以下是我提取的主要信息：

{"产品品类": "内衣", "产品品牌": "帕兰朵", "产品特点": "小胸聚拢", "受众群体": "精致妈妈人群"}


In [5]:
#多轮对话
prompt2 = """
你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容：
主题：推荐一下TriPollar的美容仪
品牌卖点：全新、护肤、清洁模式、幻白模式、紧致模式、电流感
"""
response, history = model.chat(tokenizer, prompt2, [])
print(response)

亲们，今天给大家安利一款超级好用的美容仪——TriPollar！

首先，TriPollar是全新推出的美容仪，采用最先进的技术，可以帮助我们达到美容的效果。

其次，TriPollar有护肤、清洁模式。可以有效地清洁皮肤，并且有助于预防毛孔堵塞和黑头形成。

还有幻白模式，可以有效地减少黑色素的沉淀，让我们的皮肤更加明亮均匀。

此外，TriPollar还可以帮助我们紧致皮肤，让我们的皮肤更加有弹性。

最后，TriPollar还有电流感功能，可以帮助我们更好地导入营养成分，让我们的皮肤更加健康。

总之，TriPollar是一款非常实用的美容仪，可以让我们达到美容的效果，并且帮助我们紧致皮肤，让我们的皮肤更加有弹性。如果你也想拥有一张美丽的脸，那么TriPollar绝对是你不容错过的选择！


In [5]:
#多轮对话
prompt3 = """
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类": ,"产品品牌": ,"产品特点":}
很好，把减肥群体去掉，另外突出爆款，多加点表情\n
"""
response, history = model.chat(tokenizer, prompt3, history)
print(response)

{
"产品品类": "益生菌",
"产品品牌": "汤臣倍健",
"产品特点": "爆款,改善肠道健康,促进减肥效果"
}


In [4]:
df = pd.read_excel('test_data.xlsx', engine='openpyxl')  
df = df.assign(cost_time = None, brand_name = None, class_name=None, main_point = None)
df

,title,content_clean,brandName_list,待标注关键词,cost_time,brand_name,class_name,main_point
0,谁还不是精致女孩呢,今天是护肤日 明天就要健身啦 ✨silkn 真的是我唯一的美容仪器还坚持用下来了 因为凝胶快...,['silk’n_美容仪'],护肤，拯救干燥肌肤，精致,None,None,None,None
1,宙斯反馈,去年买的宙斯，泪沟没了自己都不敢相信使用率可能想起来连着做几天，忘了就半个多月不做，既然有这...,['Dr.Arrivo_美容仪'],泪沟消失，功效显著，坚持使用，改善肤质，方便易用，持续效果，抗衰,None,None,None,None
2,雅萌好物开箱,年纪过了30+，肌肤状态明显开始走下坡路。挣扎了好久，终于狠心入手了雅萌美容仪。 收到包裹...,['雅萌YA-MAN_美容仪'],护肤，精致大气，舒适，修复，持续使用,None,None,None,None
3,想买美容仪拯救皮肤了，宙斯二代美容仪怎么,求介绍以前化妆总是很应付，卸妆更是比较随意，有时候赶紧比较累，直接带妆睡觉， 结果导致皮肤越...,['Dr.Arrivo_美容仪'],多档位调节，舒适，温和，消肿，宙斯二代，效果显著,None,None,None,None
4,过了30岁别再瞎买美容仪了！直接上初普吧！,实话实说，射频咱就得服人家以色列 妥妥射频届的老大哥！ 我人生的第一台美容仪就是tripol...,['TriPollar_美容仪'],射频美容仪，安全，资深，升级技术,None,None,None,None
...,...,...,...,...,...,...,...,...
6874,��留学，公演routine大公开�✌�,人人都可以站上舞台的嘞 果姐的公演从开始到结束都大大公开 顺便分享一些好东西给大家 维密反重...,"[""维多利亚的秘密Victoria's Secret_内衣""]",维密，反重力，小升杯，显身材,None,None,None,None
6875,� ☽ �·˚ � ₊ ·春日出游穿搭分享,出片率100%的秘密！ 五一快到啦～大家有想好去哪玩嘛！ 感觉每次出门都不知道穿什么 真的很...,['蕉内Bananain_内衣'],好的搭配，五一出行穿搭。,None,None,None,None
6876,爱慕爱生活，去繁存真，内在本真�,大家节日快乐呀～ 这是38节，我送给自己的一份精美礼物： 爱慕1993 三十周年礼盒！一套内...,['爱慕Aimer_内衣'],外穿内衣，清爽舒适，无钢圈支撑，V领无托，玫瑰花设计，爱慕30周年纪念,None,None,None,None
6877,解开束缚 女孩子都是自由肆意的！！,hello宝贝们我是麻花 今天男士止步 女士请进 敲咪咪来给姐妹们分享一下我的宝藏内衣�爱美...,"[""爱美丽imi's_内衣""]",夹心软支撑 稳定有型 柔软 手感光滑 有弹性 轻盈透气 无勒痕,None,None,None,None


In [10]:
import json 
prompt = """
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、产品特点，以json格式返回，json格式为{"产品品类": ,"产品品牌": ,"产品特点":}\n主题：推荐帕兰朵品牌内衣。品牌卖点：舒适透气，显小，柔软亲肤，老牌口碑。字数要求：200字以内。受众群体：产品受众主要是精致妈妈。营销目的：使用户看了有分享给朋友的想法或者购买欲望。在正文写作上，会使用惊叹号、表情符号等增加文章活力，运用悬念来引发读者好奇心。\n"
"""
for i in range(int(len(df))):
#for i in range(2):
    messages = []
    brand = df.iloc[i,2][2:-2].replace("_","品牌")
    key_word = df.iloc[i,3]
    s = prompt.replace(prompt[98:105], brand).replace(prompt[111:128], key_word)
    start_time = time.time()
    response ,history = model.chat(tokenizer,s, messages)
    end_time = time.time()
    response = response.replace('\n','')
    # 解析 JSON 字符串  
    if response[-1] == '.' or response[-1] == '。':
        response = response[0:-1]
    try:  
        data = json.loads(response)   
        df.iloc[i,5] = str(data['产品品牌'])
        df.iloc[i,6] = str(data['产品品类'])
        df.iloc[i,7] = str(data['产品特点'])
    except json.JSONDecodeError:  
        print(i,"解析JSON失败，跳过并继续执行其他代码。")
    response_time = end_time - start_time
    df.iloc[i,4] = response_time
    
    print(i,int(len(df)))
df

RuntimeError: CUDA error: invalid resource handle
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
prompt = """
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、品牌卖点，以json格式返回，json格式为{"产品品类": ,"产品品牌": ,"品牌卖点":}
主题：推荐一下TriPollar品牌的美容仪
品牌卖点：修复肌肤，美白，抑痘，无刺痛感
字数要求：200字之内
受众群体：主要针对敏感肌群体
使用场景：日常护理场景下
"""
response, history = model.chat(tokenizer, prompt, history=[])
print(response)

{
  "产品品类": "美容仪",
  "产品品牌": "TriPollar",
  "品牌卖点": "修复肌肤,美白,抑痘,无刺痛感"
}


In [18]:
import json 
prompt = """
请你根据以下内容以最快速度提取主要信息,只需提取到以下内容：营销产品品类、产品品牌、品牌卖点，以json格式返回，json格式为{"产品品类": ,"产品品牌": ,"品牌卖点":}
主题：推荐一下TriPollar品牌的美容仪
品牌卖点：修复肌肤，美白，抑痘，无刺痛感
字数要求：200字之内
受众群体：主要针对敏感肌群体
使用场景：日常护理场景下
"""
for i in range(300):
#for i in range(2):
    messages = []
    brand = df.iloc[i,2][2:-2].replace("_","品牌")
    key_word = df.iloc[i,3]
    s = prompt.replace(prompt[100:115], brand).replace(prompt[120:136], key_word)
    start_time = time.time()
    response ,history = model.chat(tokenizer,s, messages)
    end_time = time.time()
    response = response.replace('\n','')
    # 解析 JSON 字符串  
    if response[-1] == '.' or response[-1] == '。':
        response = response[0:-1]
    try:  
        data = json.loads(response)   
        df.iloc[i,5] = str(data['产品品牌'])
        df.iloc[i,6] = str(data['产品品类'])
        df.iloc[i,7] = str(data['品牌卖点'])
    except json.JSONDecodeError:  
        print(i,"解析JSON失败，跳过并继续执行其他代码。")
    response_time = end_time - start_time
    df.iloc[i,4] = response_time
    
    print(i,int(len(df)))
df

0 6879
1 6879
2 6879
3 解析JSON失败，跳过并继续执行其他代码。
3 6879
4 6879
5 6879
6 6879
7 6879
8 6879
9 6879
10 6879
11 6879
12 6879
13 6879
14 6879
15 6879
16 6879
17 6879
18 6879
19 6879
20 6879
21 6879
22 6879
23 6879
24 6879
25 6879
26 6879
27 6879
28 6879
29 6879
30 6879
31 6879
32 6879
33 6879
34 6879
35 6879
36 6879
37 6879
38 6879
39 6879
40 6879
41 6879
42 6879
43 6879
44 6879
45 6879
46 6879
47 6879
48 6879
49 6879
50 6879
51 6879
52 6879
53 6879
54 6879
55 解析JSON失败，跳过并继续执行其他代码。
55 6879
56 6879
57 6879
58 6879
59 6879
60 6879
61 6879
62 6879
63 6879
64 6879
65 6879
66 6879
67 6879
68 6879
69 6879
70 6879
71 6879
72 6879
73 6879
74 6879
75 6879
76 6879
77 6879
78 解析JSON失败，跳过并继续执行其他代码。
78 6879
79 6879
80 6879
81 6879
82 6879
83 6879
84 解析JSON失败，跳过并继续执行其他代码。
84 6879
85 6879
86 6879
87 6879
88 6879
89 6879
90 6879
91 解析JSON失败，跳过并继续执行其他代码。
91 6879
92 6879
93 6879
94 6879
95 6879
96 6879
97 6879
98 解析JSON失败，跳过并继续执行其他代码。
98 6879
99 6879
100 6879
101 6879
102 6879
103 6879
104 解析JSON失败，跳过并继续执行其他代码。

,title,content_clean,brandName_list,待标注关键词,cost_time,brand_name,class_name,main_point
0,谁还不是精致女孩呢,今天是护肤日 明天就要健身啦 ✨silkn 真的是我唯一的美容仪器还坚持用下来了 因为凝胶快...,['silk’n_美容仪'],护肤，拯救干燥肌肤，精致,1.041135,silk’n,美容仪,"护肤,拯救干燥肌肤,精致"
1,宙斯反馈,去年买的宙斯，泪沟没了自己都不敢相信使用率可能想起来连着做几天，忘了就半个多月不做，既然有这...,['Dr.Arrivo_美容仪'],泪沟消失，功效显著，坚持使用，改善肤质，方便易用，持续效果，抗衰,1.540559,Dr.Arrivo,美容仪,泪沟消失，功效显著，坚持使用，改善肤质，方便易用，持续效果，抗衰
2,雅萌好物开箱,年纪过了30+，肌肤状态明显开始走下坡路。挣扎了好久，终于狠心入手了雅萌美容仪。 收到包裹...,['雅萌YA-MAN_美容仪'],护肤，精致大气，舒适，修复，持续使用,1.043488,雅萌YA-MAN,美容仪,"护肤,精致大气,舒适,修复,持续使用"
3,想买美容仪拯救皮肤了，宙斯二代美容仪怎么,求介绍以前化妆总是很应付，卸妆更是比较随意，有时候赶紧比较累，直接带妆睡觉， 结果导致皮肤越...,['Dr.Arrivo_美容仪'],多档位调节，舒适，温和，消肿，宙斯二代，效果显著,1.036385,None,None,None
4,过了30岁别再瞎买美容仪了！直接上初普吧！,实话实说，射频咱就得服人家以色列 妥妥射频届的老大哥！ 我人生的第一台美容仪就是tripol...,['TriPollar_美容仪'],射频美容仪，安全，资深，升级技术,0.855558,TriPollar,美容仪,射频美容仪，安全，资深，升级技术
...,...,...,...,...,...,...,...,...
6874,��留学，公演routine大公开�✌�,人人都可以站上舞台的嘞 果姐的公演从开始到结束都大大公开 顺便分享一些好东西给大家 维密反重...,"[""维多利亚的秘密Victoria's Secret_内衣""]",维密，反重力，小升杯，显身材,None,None,None,None
6875,� ☽ �·˚ � ₊ ·春日出游穿搭分享,出片率100%的秘密！ 五一快到啦～大家有想好去哪玩嘛！ 感觉每次出门都不知道穿什么 真的很...,['蕉内Bananain_内衣'],好的搭配，五一出行穿搭。,None,None,None,None
6876,爱慕爱生活，去繁存真，内在本真�,大家节日快乐呀～ 这是38节，我送给自己的一份精美礼物： 爱慕1993 三十周年礼盒！一套内...,['爱慕Aimer_内衣'],外穿内衣，清爽舒适，无钢圈支撑，V领无托，玫瑰花设计，爱慕30周年纪念,None,None,None,None
6877,解开束缚 女孩子都是自由肆意的！！,hello宝贝们我是麻花 今天男士止步 女士请进 敲咪咪来给姐妹们分享一下我的宝藏内衣�爱美...,"[""爱美丽imi's_内衣""]",夹心软支撑 稳定有型 柔软 手感光滑 有弹性 轻盈透气 无勒痕,None,None,None,None


In [19]:
df[0:300]

,title,content_clean,brandName_list,待标注关键词,cost_time,brand_name,class_name,main_point
0,谁还不是精致女孩呢,今天是护肤日 明天就要健身啦 ✨silkn 真的是我唯一的美容仪器还坚持用下来了 因为凝胶快...,['silk’n_美容仪'],护肤，拯救干燥肌肤，精致,1.041135,silk’n,美容仪,"护肤,拯救干燥肌肤,精致"
1,宙斯反馈,去年买的宙斯，泪沟没了自己都不敢相信使用率可能想起来连着做几天，忘了就半个多月不做，既然有这...,['Dr.Arrivo_美容仪'],泪沟消失，功效显著，坚持使用，改善肤质，方便易用，持续效果，抗衰,1.540559,Dr.Arrivo,美容仪,泪沟消失，功效显著，坚持使用，改善肤质，方便易用，持续效果，抗衰
2,雅萌好物开箱,年纪过了30+，肌肤状态明显开始走下坡路。挣扎了好久，终于狠心入手了雅萌美容仪。 收到包裹...,['雅萌YA-MAN_美容仪'],护肤，精致大气，舒适，修复，持续使用,1.043488,雅萌YA-MAN,美容仪,"护肤,精致大气,舒适,修复,持续使用"
3,想买美容仪拯救皮肤了，宙斯二代美容仪怎么,求介绍以前化妆总是很应付，卸妆更是比较随意，有时候赶紧比较累，直接带妆睡觉， 结果导致皮肤越...,['Dr.Arrivo_美容仪'],多档位调节，舒适，温和，消肿，宙斯二代，效果显著,1.036385,None,None,None
4,过了30岁别再瞎买美容仪了！直接上初普吧！,实话实说，射频咱就得服人家以色列 妥妥射频届的老大哥！ 我人生的第一台美容仪就是tripol...,['TriPollar_美容仪'],射频美容仪，安全，资深，升级技术,0.855558,TriPollar,美容仪,射频美容仪，安全，资深，升级技术
...,...,...,...,...,...,...,...,...
295,雅萌MAX二代,李老头推荐雅萌max二代多功能旗舰射频仪提亮美容仪脸部 \t 作业 盒豚生活 领减 2600...,['雅萌YA-MAN_美容仪'],提亮，多功能,0.822568,雅萌YA-MAN,美容仪,提亮，多功能，适合敏感肌群体，日常护理场景下使用
296,抗老的尽头是初普！✅不能错过的射频仪 ！,❤️一过25岁，我的护肤品就开始没有多大效果了，这时候尝试了美容仪，发现效果真的很明显！ ❤...,['TriPollar_美容仪'],抗老，嘭弹紧致，效果明显,0.824818,TriPollar,美容仪,抗老，膨弹紧致，效果明显
297,怪不得你的美容仪没用，千万不能忽略这点！,皮肤衰老不仅仅是真皮胶原流失那么简单，还包括皮下脂肪下垂萎缩和筋膜层松弛️ 所以注重分层抗考...,['松下Panasonic_美容仪'],多维度抗老，强射频促胶原，双重EMS电流活动肌肉，IP电离子导入增强护肤品渗透,0.977811,松下Panasonic,美容仪,多维度抗老，强射频促胶原，双重EMS电流活动肌肉，IP电离子导入增强护肤品渗透
298,雅萌MAX二代多功能旗舰射频仪,纠结了很多品牌，最后趁着活动有优惠就买了，还送了很多礼物，看了使用教程很简单一学就会，好上手...,['雅萌YA-MAN_美容仪'],多功能滋润皮肤，有效使用，赠品多，操作简单,0.776151,雅萌YA-MAN,美容仪,多功能滋润皮肤，有效使用，赠品多，操作简单


In [20]:
output_file = 'output.xlsx'  
df[0:300].to_excel(output_file, index=False)

In [23]:
df_result = pd.read_excel("output.xlsx")
df_result.describe()

,cost_time
count,300.000000
mean,1.019302
std,0.334571
min,0.512150
25%,0.819074
50%,0.978300
75%,1.150805
max,4.382658


In [26]:
if df_result.iloc[3,5] :
    print(0)
else:
    print(1)

0
